In [12]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from sklearn.cluster import KMeans
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load your cleaned dataframe
df = pd.read_csv("fitzpatrick17k.csv")
df = df.dropna(subset=['url']).reset_index(drop=True)

# Filter only Psoriasis images
psoriasis_df = df[df['label'].str.lower() == 'psoriasis']
os.makedirs("images/psoriasis", exist_ok=True)


In [16]:
unique_labels = df['label'].unique()
label_counts = df['label'].value_counts()
print(label_counts)


label
psoriasis                      653
squamous cell carcinoma        581
lichen planus                  491
basal cell carcinoma           468
allergic contact dermatitis    430
                              ... 
port wine stain                 59
erythema elevatum diutinum      55
xanthomas                       53
pustular psoriasis              53
pilomatricoma                   53
Name: count, Length: 114, dtype: int64


In [27]:
# Downloading all of the Psoriasis images from the url field

def download_image(url, save_path):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            try:
                image = Image.open(BytesIO(response.content)).convert("RGB")
                image.save(save_path)
                return True
            except Exception as e:
                print(f"Invalid image at {url} | {e}")
        else:
            print(f"Bad status ({response.status_code}) for {url}")
    except Exception as e:
        print(f"Failed to download {url} | {e}")
    return False


# Download all Psoriasis images
for idx, row in psoriasis_df.iterrows():
    img_name = f"images/psoriasis/psoriasis_{idx}.jpg"
    if not os.path.exists(img_name):
        download_image(row['url'], img_name)


Number of files in 'images/psoriasis': 1959


In [ ]:
#############################################################################################################################################

In [ ]:
##########################################################